Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# 3D regression example based on DenseNet

This tutorial shows an example of 3D regression task based on DenseNet and array format transforms.

Here, the task is given to predict the ages of subjects from MR imagee.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/3d_regression/densenet_training_array.ipynb)

## Setup environment

In [2]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'


## Setup imports

In [2]:
import logging
import os
import sys
import shutil
import tempfile

import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np

import monai
from monai.config import print_config
from monai.transforms import (
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)
from monai.networks.nets import Regressor
from torch.utils.data import Dataset, DataLoader  # 加入 Dataset 用於自定義數據處理

# 檢查是否支援 CUDA
pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 配置日誌輸出
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

# 自定義 Dataset 以處理 .npz 文件
class NpzDataset(Dataset):
    def __init__(self, file_paths, labels, key="z_mu", transform=None):
        """
        :param file_paths: .npz 文件路徑列表
        :param labels: 對應的標籤列表
        :param key: 要讀取的鍵名 (默認為 'z_mu')
        :param transform: 數據增強或預處理的變換
        """
        self.file_paths = file_paths
        self.labels = labels
        self.key = key
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 加載 .npz 文件並提取指定鍵的數據
        file_path = self.file_paths[idx]
        with np.load(file_path) as data:
            image = data[self.key]  # 提取鍵 z_mu 的數據
            # 去掉額外維度，例如 (1, 16, 32, 32, 18) -> (16, 32, 32, 18)
            if image.ndim == 5 and image.shape[0] == 1:
                image = np.squeeze(image, axis=0)
        label = self.labels[idx]

        # 如果有變換，對數據進行處理
        if self.transform:
            image = self.transform(image)

        return image, label


MONAI version: 1.5.dev2445
Numpy version: 1.26.2
Pytorch version: 2.5.1+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 2af9926d853086b264680adcf954bf3232f5ec32
MONAI __file__: c:\Users\<username>\AppData\Local\Programs\Python\Python311\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.11.4
Pillow version: 10.1.0
Tensorboard version: 2.18.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.4
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.0.0
pandas version: 2.1.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED

## Setup data directory

In [3]:
# Set data directory
directory = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI'
os.makedirs(directory, exist_ok=True)
root_dir = directory

print(root_dir)

C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI


In [6]:
import os
import pandas as pd
import numpy as np
from glob import glob

# 讀取 Excel 檔案
excel_file = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI\MONAI_trainval.xlsx'
df = pd.read_excel(excel_file)

# 根目錄與子資料夾
nii_dir = r'C:\Users\Hikari20220126i712th\Downloads\IXI-T1\MONAI\outputencode'

# 初始化影像路徑和年齡清單
images = []
ages = []

# 從 Excel 文件中動態匹配影像檔案
for _, row in df.iterrows():
    file_id = row["IXI_ID"]  # 從 Excel 中獲取 ID（如 "IXI002"）
    age = row["AGE"]  # 從 Excel 中獲取對應年齡

    # 根據 ID 搜索對應的已編碼壓縮檔*.npz
    search_pattern = os.path.join(nii_dir, f"{file_id}*.npz")
    matching_files = glob(search_pattern)

    # 如果找到對應的 *.npz 檔案，加入清單
    if len(matching_files) > 0:
        file_path = matching_files[0]
        with np.load(file_path) as data:
            # 提取指定鍵名 'z_mu'
            if 'z_mu' in data.files:
                image_data = data['z_mu']
                # 如果影像數據的第一維是 1，則刪除該維度 (從 [1, 16, 32, 32, 18] 到 [16, 32, 32, 18])
                if image_data.ndim == 5 and image_data.shape[0] == 1:
                    image_data = np.squeeze(image_data, axis=0)
                images.append(image_data)
            else:
                print(f"'z_mu' key not found in {file_path}")
        ages.append(age)  # 添加對應的年齡
    else:
        print(f"File not found for ID: {file_id}")

# 將年齡轉為 NumPy 陣列，與原始程式保持一致
ages = np.array(ages)

# 確認資料載入成功
print(f"Loaded {len(images)} images and {len(ages)} age labels.")


Loaded 446 images and 446 age labels.


## Create data loaders

In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from monai.transforms import Compose, ScaleIntensity, Resize, RandRotate90
import numpy as np
import torch
import time

# 確認是否有 GPU 支援
pin_memory = torch.cuda.is_available()

# 確保數據具有正確的維度 (16, 32, 32, 18)
for i in range(len(images)):
    if images[i].ndim == 4 and images[i].shape[0] == 1:  # 如果形狀是 [1, 16, 32, 32, 18]
        images[i] = np.squeeze(images[i], axis=0)  # 移除第一維度，變為 [16, 32, 32, 18]

# 定義資料轉換
train_transforms = Compose([
    ScaleIntensity(),  # 將數據標準化到 [0, 1] 範圍
    Resize(spatial_size=(32, 32, 18)),  # 確保影像大小正確
    RandRotate90()  # 隨機旋轉
])

val_transforms = Compose([
    ScaleIntensity(),
    Resize(spatial_size=(32, 32, 18))  # 確保驗證影像大小一致
])

# 使用 train_test_split 分割資料，70% 為訓練集，30% 為驗證集
train_images, val_images, train_ages, val_ages = train_test_split(
    images, ages, test_size=0.18, random_state=42
)

# 檢查資料分割結果
print(f"Training samples: {len(train_images)}, Validation samples: {len(val_images)}")

# 定義自定義 Dataset 類
class NpzDataset(Dataset):
    def __init__(self, image_data, labels, transform=None):
        """
        自定義 Dataset，處理內存中的 Numpy 數據
        :param image_data: Numpy array 列表
        :param labels: 對應的標籤列表
        :param transform: MONAI 的數據轉換
        """
        self.image_data = image_data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        image = self.image_data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# 定義訓練資料集與 DataLoader
train_ds = NpzDataset(image_data=train_images, labels=train_ages, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0, pin_memory=pin_memory)

# 定義驗證資料集與 DataLoader
val_ds = NpzDataset(image_data=val_images, labels=val_ages, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0, pin_memory=pin_memory)

# 測試 DataLoader 的第一個批次
check_ds = NpzDataset(image_data=images, labels=ages, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=3, num_workers=0, pin_memory=pin_memory)

# 測試 DataLoader 加載效率
start_time = time.time()

for idx, (im, label) in enumerate(check_loader):
    print(f"Batch {idx+1}: Inputs shape {im.shape}, Labels shape {label.shape}")
    if idx == 2:  # 測試前三批次
        break

end_time = time.time()
print(f"Time for loading 3 batches: {end_time - start_time:.2f} seconds")



Training samples: 365, Validation samples: 81
Batch 1: Inputs shape torch.Size([3, 16, 32, 32, 18]), Labels shape torch.Size([3])
Batch 2: Inputs shape torch.Size([3, 16, 32, 32, 18]), Labels shape torch.Size([3])
Batch 3: Inputs shape torch.Size([3, 16, 32, 32, 18]), Labels shape torch.Size([3])
Time for loading 3 batches: 0.01 seconds


## Create model and train

In [ ]:
model = Regressor(in_shape=[16, 32, 32, 18], out_shape=1, channels=(16, 32, 64, 128, 256), strides=(2, 2, 2, 2))#修改維度
if torch.cuda.is_available():
    model.cuda()
# It is important that we use nn.MSELoss for regression.
loss_function = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 3457  # 修改 epochs 數

lowest_rmse = sys.float_info.max
for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        all_labels = []
        all_val_outputs = []
        for val_data in val_loader:
            val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
            all_labels.extend(val_labels.cpu().detach().numpy())
            with torch.no_grad():
                val_outputs = model(val_images)
                flattened_val_outputs = [val for sublist in val_outputs.cpu().detach().numpy() for val in sublist]
                all_val_outputs.extend(flattened_val_outputs)

        mse = np.square(np.subtract(all_labels, all_val_outputs)).mean()
        rmse = np.sqrt(mse)

        if rmse < lowest_rmse:
            lowest_rmse = rmse
            lowest_rmse_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_array.pth")
            print("saved new best metric model")

        print(f"Current epoch: {epoch+1} current RMSE: {rmse:.4f} ")
        print(f"Best RMSE: {lowest_rmse:.4f} at epoch {lowest_rmse_epoch}")
        writer.add_scalar("val_rmse", rmse, epoch + 1)

print(f"Training completed, lowest_rmse: {lowest_rmse:.4f} at epoch: {lowest_rmse_epoch}")
writer.close()


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #"last_expr" 


## Cleanup data directory

Remove directory if a temporary was used.

In [7]:
if directory is None:
    shutil.rmtree(root_dir)